In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
#path
path_func = '/media/data/Dunedin_Study_Data_Narun_P_Jan2022/DBIS_NarunPat_sharedJan2022/fMRI_Regions.N379/'
path_str = '/media/data/Dunedin_Study_Data_Narun_P_Jan2022/DBIS_NarunPat_sharedJan2022/'
path_mov = '/media/data/Dunedin_Study_Data_Narun_P_Jan2022/DBIS_NarunPat_sharedJan2022_QC/QC/MotionFiles/'

In [3]:
#labels
labels379 = pd.read_csv(path_func+'FC_labels.csv', header=None).values.ravel()
#display(labels379)

In [14]:
#load data and calc cor matx, flatten them
folders = [i.split('/')[-1] for i in sorted(glob.glob(path_func+'FC_*_matrices'))[2:]]

dct_task_fc={}

for folder in folders:
    
    subjects = [i[:-4] for i in sorted(os.listdir(path_func+folder+'/'))]

    dct_group_fc={}

    for subject in subjects:
        df = pd.read_csv(path_func+folder+'/'+subject+'.csv', header=None)
        df.index, df.columns = labels379, labels379
        #r-2-z
        df = np.arctanh(df)
        #convert to string
        dfg = df.where(np.triu(np.ones(df.shape), k=1).astype(bool)).stack().reset_index()
        indx = [i+'_&_'+j for i,j in zip(dfg['level_0'], dfg['level_1'])]
        fc_string = pd.Series(np.array(dfg[0]),index=indx)
        #write a row to dict
        dct_group_fc[subject] = fc_string

    dct_task_fc[folder.split('_')[1]] = pd.DataFrame(dct_group_fc).T



In [19]:
#check for nan
for key in dct_task_fc.keys():
    print(key)
    print(dct_task_fc[key].isna().sum()[dct_task_fc[key].isna().sum()>0])

facename
Series([], dtype: int64)
faces
Series([], dtype: int64)
mid
Series([], dtype: int64)
stroop
Series([], dtype: int64)


In [17]:
#load main part indexes and check for common
indx_ml = pd.read_csv('/media/data/Dunedin_Study_Data_Narun_P_Jan2022/MLTabs/info.csv', index_col=0).index
display(indx_ml)

Index(['sub-0201', 'sub-0202', 'sub-0203', 'sub-0205', 'sub-0208', 'sub-0210',
       'sub-0211', 'sub-0213', 'sub-0214', 'sub-0215',
       ...
       'sub-1225', 'sub-1226', 'sub-1228', 'sub-1230', 'sub-1231', 'sub-1233',
       'sub-1234', 'sub-1235', 'sub-1236', 'sub-1237'],
      dtype='object', length=754)

In [18]:
for key in dct_task_fc.keys():
    dct_task_fc[key] = dct_task_fc[key].reindex(index=indx_ml)

In [ ]:
for key in dct_task_fc.keys():
    print(key)
    display(dct_task_fc[key])

In [21]:
#save
for key in dct_task_fc.keys():
    dct_task_fc[key].to_csv('/media/data/Dunedin_Study_Data_Narun_P_Jan2022/MLTabs/'+key+'_FC.csv')